In [13]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from PIL import Image

from core.integrate import FeatureFusionScalableTSDFVolume, PanopticFusionScalableTSDFVolume, ScalableTSDFVolume

draw = o3d.visualization.EV.draw

In [15]:
device = 'cuda:0'

In [16]:
tsdf_device = "cuda:1"
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)

save_dir = "/home/quanta/Experiments/feature-instance-fusion/debug/"
tsdf_volume.load(save_dir + "tsdf/tsdf_volume.pt")

In [17]:
verts, faces = tsdf_volume.extract_mesh()
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)
mesh.compute_vertex_normals()

TriangleMesh with 56858 points and 105967 triangles.

In [16]:
tsdf_volume.reset_feature(dim=3, include_var=False, dtype=torch.float)
tsdf_volume.load_feats(save_dir + "tsdf_feature_color/feats.pt")

In [19]:
color = tsdf_volume.extract_feat_on_grid(verts)[0]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)
draw([mesh])

In [18]:
tsdf_device = "cuda:1"
tsdf_volume = PanopticFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device=tsdf_device,
)
tsdf_volume.load(save_dir + "tsdf/tsdf_volume.pt")
tsdf_volume.reset_instance()
# tsdf_volume.load_instance_from_path(save_dir + "tsdf_panoptic_grounded_sam/panoptic_labels.pt")
tsdf_volume.load_instance_from_path(save_dir + "tsdf_panoptic_mask_rcnn/panoptic_labels.pt")

In [19]:
labels = tsdf_volume.extract_label_on_grid(verts)[0]
color_set = np.random.random(size=(tsdf_volume._instance_label_num, 3))
color = color_set[labels]
mesh.vertex_colors = o3d.utility.Vector3dVector(color)
draw([mesh])